# Dataset Transforms Example

Transforms can be applied to PIL images, tensors, ndarrays, or custom data during creatio of the dataset. Complete list of built-in transforms: https://pytorch.org/vision/stable/transforms.html

On Images:
- CenterCrop, Grayscale, Pad, etc.

On Tensors:
- LinearTransformation, Normalize, RandomErasing, etc.

Conversion:
- ToPILImage: from tensor or ndarray
- ToTensorL from numpy.ndarray or PILImage

Generic:
- Use lambda

Custom:
- Write custom class

Compose multiple transforms:
ex. `composed = transforms.Compose([Rescale(256), RandomCrop(224)])`

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [2]:
class WineDataset(Dataset):

    def __init__(self, transform=None):
        # Initialize data, download, etc.
        # read with numpy or pandas
        xy = np.loadtxt('data/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]
        self.transform = transform
        
        # here the first column is the class label, the rest are the features
        self.x_data = xy[:, 1:] # size [n_samples, n_features]
        self.y_data = xy[:, [0]] # size [n_samples, 1]

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample
    
    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples
    
    
class ToTensor:
       
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)
    

class MulTransform:
    
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, sample):
        inputs, targets = sample
        inputs = inputs * self.factor
        return inputs, targets

In [3]:
dataset = WineDataset(transform=ToTensor())

In [4]:
first_data = dataset[0]
features, labels = first_data

print(type(features), type(labels))
print(features)

<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03])


In [5]:
factor = 2
compose = torchvision.transforms.Compose([ToTensor(), MulTransform(factor)])

dataset = WineDataset(transform=compose)

first_data = dataset[0]
features, labels = first_data

print(type(features), type(labels))
print(features)

<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03])
